<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%9D%D0%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Нейронные сети
####Вариант 5:
Просчитать одну итерацию цикла обучения по Δ-правилу однослойной бинарной
неоднородной нейронной сети, состоящей из 2 нейронов и имеющей функции активации: гиперболический тангенс ($k=2$) и пороговую функцию ($Т=0.5$). В качестве обучающей выборки использовать таблицу истинности для операций эквивалентности и конъюнкции (не использовать первую строчку таблицы). Синаптические веса задать случайным образом.

In [1]:
import numpy as np
import pandas as pd
import random
import copy
import math
import itertools as it

In [9]:
#функции активации
#гиперболический тангенс:
def hyperbolicTangentFunction(S, k = 2):
  return math.tanh(S / k)
#сигмоидная функция:
def sigmoidFunction(S, k = 2):
  return 1.0 / (1 + math.exp(- S * k))
#Пороговая функция:
def thresholdFunction(S, T = 0.5):
  return 1 if S >= T else 0

#=================================================================
def randomVector(size = 2):
  return [random.uniform(0, 1) for i in range(size)]

def randomNumber():
  return random.uniform(0, 1)

def equiv(a,b): # эквивалентность ~
  rez = 0
  if((a == 1 and b == 1) or (a == 0 and b == 0)): rez = 1
  return rez

#=================================================================
#функции для формирования таблицы данных
def x1(x, v):
  return math.sin(math.log(abs(v[0])) / x[0])

def x2(x, v):
  return math.cos(v[1]**3)

def x3(x, v):
  return math.sin(math.tan(x[0] / x[1]) - 3**(v[2]))

def v1(x, v):
  return (v[0] + math.tan(x[0])**2)

def v2(x, v):
  return (0.2 * v[1])

def v3(x, v):
  return (math.exp(x[0]) + .45 * math.log2(abs(x[1] * v[2])))

def y1(x, v):
  return (x[0] / v[0]) * math.sqrt(abs(x[0]))

def y2(x, v):
  return math.sqrt(abs(math.sin(v[1])))

#def y3(x, v):
  #return v[2] * x[0] / (x[1] + 0.1)

inputFunctions = [x1, x2, x3, v1, v2, v3]
outputFunctions = [y1, y2]
#=================================================================
#Генерация случайного вектора из 3-х элементов в диапазоне от -3 до 3
def generationRandomVector(elem = 3, start = -3, end = 3):
  vector = []
  for i in range(elem):
    vector.append(random.uniform(start, end))
  return vector

#=================================================================
#создание столбца таблицы
def createVariants(function, elem = 30):
  vector = []
  for i in range(elem):
    vector.append(function(generationRandomVector(), generationRandomVector()))
  return vector

#подставление x,v в выходные функции
def addVariants(function, x, v, elem = 30):
  vector = []
  for i in range(elem):
    vector.append(function(x[i], v[i]))
  return vector

In [11]:
#Коэффициент обучения (точность) [0;1]
learningRate = 0.8
#Количество слоёв
layer = 1
#количество нейронов на каждом слое
#neurons = [2,3,1]
neurons = 2
#количество входов/выходов соответственно
countX = 2
countY = 2

countV = 3

networkCharact = [layer, neurons, learningRate]
inpOutCounts = [countX, countY]

#=================================================================
data = []
for i in range(len(inputFunctions)):
  data.append(createVariants(inputFunctions[i]))
for i in range(len(outputFunctions)):
  data.append(addVariants(outputFunctions[i], np.array(data[0:3]).T, np.array(data[3:]).T))

data_df = pd.DataFrame(np.array(data).T, columns=['x1', 'x2', 'x3', 'v1', 'v2', 'v3', 'y1', 'y2'])

In [12]:
data_df

,x1,x2,x3,v1,v2,v3,y1,y2
0,-0.024914,0.995759,0.983417,5422.591500,0.142021,1.751884,-7.252003e-07,0.376224
1,0.998614,-0.977893,-0.939296,2.414867,-0.345967,1.774603,4.132412e-01,0.582329
2,0.057827,0.999905,-0.953559,-0.346173,-0.163319,0.882177,-4.017050e-02,0.403230
3,-0.019686,0.822749,0.396166,3.190579,0.180627,-0.599587,-8.657220e-04,0.423847
4,-0.749090,0.983402,0.345738,-2.870937,-0.459513,3.308050,2.258277e-01,0.665966
5,0.360478,0.925681,-0.942041,0.088372,-0.480052,0.745853,2.449075e+00,0.679577
6,-0.355780,0.999959,-0.873183,173.696271,-0.558690,9.978791,-1.221747e-03,0.728063
7,-0.089940,-0.249192,-0.944698,-1.957014,-0.145743,8.915447,1.378283e-02,0.381088
8,0.008616,-0.992588,-0.791091,1.608017,0.222794,12.383698,4.973421e-04,0.470059
9,-0.544880,-0.195382,-0.864016,0.331816,0.143885,3.882377,-1.212143e+00,0.378667


In [ ]:
#charactNet = [кол-во слоёв, кол-во нейронов, коэф. обучения]
#sizeSystem = [кол-во входов, кол-во выходов]

def deltaRule(function, data, charactNet, sizeSystem):
  countLayers = charactNet[0]
  countNeurons = charactNet[1]
  nabla = charactNet[2]

  indexOrder = [i for i in range(0, len(data))] #список всех индексов строк таблицы данных

  si = [0] * sizeSystem[1]
  yi = [0] * sizeSystem[1]
  epsi = [0] * sizeSystem[1]
  level_i = 0
  k = 0

  while(level_i < charactNet[0]): #цикл по слоям
    wi = [[randomNumber() for i in range(sizeSystem[0])] for j in range(charactNet[1])] #синаптические веса

    print("\tЭпоха №" + str(level_i + 1))
    random.shuffle(indexOrder)
    while(k < len(data)): #цикл по строкам таблицы с данными
      eps = 0
      print("Матрица весов для " + str(k + 1) + "-й итерации:")
      print(pd.DataFrame(wi))
      print("\nДля X = ( " + str(data[data.columns[0]][indexOrder[k]]) + ", " + str(data[data.columns[1]][indexOrder[k]]) + ' )')

      for i in range(charactNet[1]): #по нейронам
        for j in range(sizeSystem[0]): #по входам
          si[i] += data[data.columns[j]][indexOrder[k]] * wi[j][i]

        #сделать проверку на сравнение количества нейронов и выходов для последнего слоя!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        yi[i] = function(si[i])
        epsi[i] = data[data.columns[sizeSystem[1] + i]][indexOrder[k]] - yi[i]
        eps += epsi[i]**2

      rez_i = pd.DataFrame([si, yi, epsi], index=['S', 'Y расч.', 'EPS'], columns=['neuron ' + str(i+1) for i in range(sizeSystem[1])])
      print(rez_i)
      print("\nСКО: " + str(eps) + "\n\n")

      #вычисление весов для следующей итерации:
      for i in range(charactNet[1]):
        for j in range(sizeSystem[0]):
          wi[j][i] = wi[j][i] - nabla * epsi[i] * data[data.columns[j]][indexOrder[k]]

      k += 1

    level_i += 1
    k = 0

  #

In [ ]:
table = pd.DataFrame(it.product([0,1], repeat=countX), columns=['X' + str(i + 1) for i in range(countX)])
table['Y1 = equivalence (~)'] = [equiv(table[table.columns[0]][i], table[table.columns[1]][i]) for i in range(2**neurons)]
table['Y2 = conjunction (^)'] = [(table[table.columns[0]][i] and table[table.columns[1]][i]) for i in range(2**neurons)]

deltaRule(hyperbolicTangentFunction, table, networkCharact, inpOutCounts)
#deltaRule(sigmoidFunction, table, layer, neurons, learningRate)
#deltaRule(thresholdFunction, table, layer, neurons, learningRate)

	Эпоха №1
Матрица весов для 1-й итерации:
          0         1
0  0.692479  0.505371
1  0.391571  0.685794

Для X = ( 0, 0 )
         neuron 1  neuron 2
S             0.0       0.0
Y расч.       0.0       0.0
EPS           1.0       0.0

СКО: 1.0


Матрица весов для 2-й итерации:
          0         1
0  0.692479  0.505371
1  0.391571  0.685794

Для X = ( 0, 1 )
         neuron 1  neuron 2
S        0.391571  0.685794
Y расч.  0.193322  0.330061
EPS     -0.193322 -0.330061

СКО: 0.1463135725241173


Матрица весов для 3-й итерации:
          0         1
0  0.692479  0.505371
1  0.546228  0.949843

Для X = ( 1, 0 )
         neuron 1  neuron 2
S        1.084050  1.191164
Y расч.  0.494519  0.533898
EPS     -0.494519 -0.533898

СКО: 0.5295967683166428


Матрица весов для 4-й итерации:
          0         1
0  1.088094  0.932489
1  0.546228  0.949843

Для X = ( 1, 1 )
         neuron 1  neuron 2
S        2.718372  3.073496
Y расч.  0.876204  0.911572
EPS      0.123796  0.088428

СКО: 0.0231

In [ ]:
#1-Количество входов и выходов могут быть различны +
#2-Реализовать расчёт промежуточных слоёв

In [ ]:
#Характеристики многослойной сети:
#1- вектор из кол-ва нейронов в каждом слое (в каждом слое м/б разное кол-во нейронов)
#2- кол-во выходов = кол-ву нейронов в крайнем слое
#3- входов может быть сколько угодно +
#4- в цикле: wij: i-номер нейрона, j-номер входа +
#5- размер s, eps равен количеству выходов +

In [ ]:
#1-Добавить условие для последнего слоя, где сравнивается кол-во нейронов и кол-во выходов
#2-